In [7]:
#!pip install transformers
#!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import torch
from transformers import Trainer, TrainingArguments, DefaultFlowCallback, AdamW, get_constant_schedule
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
import json
def read_json(input_file):
    """Reads a json list file."""
    with open(input_file, "r",encoding='utf-8') as f:
        reader = f.readlines()#每一行作为一个元素储存在一个list中
    return [json.loads(line.strip()) for line in reader]#json数据处理，返回列表

In [25]:
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [26]:

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [47]:
# Input text
text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

# Tokenize and encode text using batch_encode_plus
# The function returns a dictionary containing the token IDs and attention masks
encoding = tokenizer.batch_encode_plus(
    [text],                    # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)

input_ids = encoding['input_ids']  # Token IDs
# print input IDs
print(f"Input ID: {input_ids}")
attention_mask = encoding['attention_mask']  # Attention mask
# print attention mask
print(f"Attention mask: {attention_mask}")

Input ID: tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,  1010,  1996,
          2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,  2314,  2924,
          1012,   102]])
Attention mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [49]:
# Decode the token IDs back to text
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print decoded text
print(f"Decoded Text: {decoded_text}")
# Tokenize the text again for reference
tokenized_text = tokenizer.tokenize(decoded_text)
#print tokenized text
print(f"tokenized Text: {tokenized_text}")
# Encode the text
encoded_text = tokenizer.encode(text, return_tensors='pt')  # Returns a tensor
# Print encoded text
print(f"Encoded Text: {encoded_text}")

Decoded Text: after stealing money from the bank vault, the bank robber was seen fishing on the mississippi river bank.
tokenized Text: ['after', 'stealing', 'money', 'from', 'the', 'bank', 'vault', ',', 'the', 'bank', 'robber', 'was', 'seen', 'fishing', 'on', 'the', 'mississippi', 'river', 'bank', '.']
Encoded Text: tensor([[  101,  2044, 11065,  2769,  2013,  1996,  2924, 11632,  1010,  1996,
          2924, 27307,  2001,  2464,  5645,  2006,  1996,  5900,  2314,  2924,
          1012,   102]])


In [89]:
len(tokenized_text)

20

In [91]:
encoded_text.size()

torch.Size([1, 22])

In [92]:
tuple_0 = []
for i in range(len(tokenized_text)):
  tuple_0.append([tokenized_text[i],encoded_text[0][i+1]])

In [73]:
import numpy as np

In [ ]:
import torch
from torch import Tensor

cosi = torch.nn.CosineSimilarity(dim=0)

In [93]:
for idx,tuple_1 in enumerate(tuple_0):
  for idx_1,tuple_2 in enumerate(tuple_0):
    #print(idx,idx_1)
    if idx<idx_1:
      print(tuple_1[0],tuple_2[0],tuple_1[1],tuple_2[1],cosi(model.embeddings.word_embeddings.weight[tuple_1[1]],model.embeddings.word_embeddings.weight[tuple_2[1]]))


after stealing tensor(2044) tensor(11065) tensor(0.1569, grad_fn=<SumBackward1>)
after money tensor(2044) tensor(2769) tensor(0.1825, grad_fn=<SumBackward1>)
after from tensor(2044) tensor(2013) tensor(0.3969, grad_fn=<SumBackward1>)
after the tensor(2044) tensor(1996) tensor(0.3210, grad_fn=<SumBackward1>)
after bank tensor(2044) tensor(2924) tensor(0.1457, grad_fn=<SumBackward1>)
after vault tensor(2044) tensor(11632) tensor(0.1332, grad_fn=<SumBackward1>)
after , tensor(2044) tensor(1010) tensor(0.3480, grad_fn=<SumBackward1>)
after the tensor(2044) tensor(1996) tensor(0.3210, grad_fn=<SumBackward1>)
after bank tensor(2044) tensor(2924) tensor(0.1457, grad_fn=<SumBackward1>)
after robber tensor(2044) tensor(27307) tensor(0.1053, grad_fn=<SumBackward1>)
after was tensor(2044) tensor(2001) tensor(0.2844, grad_fn=<SumBackward1>)
after seen tensor(2044) tensor(2464) tensor(0.1842, grad_fn=<SumBackward1>)
after fishing tensor(2044) tensor(5645) tensor(0.1291, grad_fn=<SumBackward1>)
afte

'\n# Print word embeddings for each token\nfor token, embedding in zip(tokenized_text, word_embeddings[0]):\n    #print(f"Token: {token}")\n    print(f"Embedding: {embedding}")\n    print("\n")\nwith torch.no_grad():\n    outputs = model(input_ids, attention_mask=attention_mask)\n    word_embeddings = outputs.last_hidden_state  # This contains the embeddings\n \n# Output the shape of word embeddings\nprint(f"Shape of Word Embeddings: {word_embeddings.shape}")\n'

In [101]:
tuple_0_0 =[[tokenized_text[i],encoded_text[0][i+1],word_embeddings[0][i+1]] for i in range(len(tokenized_text))]

In [94]:
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings

# Output the shape of word embeddings
print(f"Shape of Word Embeddings: {word_embeddings.shape}")

Shape of Word Embeddings: torch.Size([1, 22, 768])


In [100]:
word_embeddings.shape

torch.Size([1, 22, 768])

In [103]:
# Print word embeddings for each token
'''
for token, embedding in zip(tokenized_text, word_embeddings[0]):
    print(f"Token: {token}")
    print(f"Embedding: {embedding}")
    print("\n")
'''
for idx,tuple_1 in enumerate(tuple_0_0):
  for idx_1,tuple_2 in enumerate(tuple_0_0):
    #print(idx,idx_1)
    if idx<idx_1:
      print(tuple_1[0],tuple_2[0],tuple_1[1],tuple_2[1],idx,idx_1,cosi(tuple_1[2],tuple_2[2]))


after stealing tensor(2044) tensor(11065) 0 1 tensor(0.4903)
after money tensor(2044) tensor(2769) 0 2 tensor(0.3421)
after from tensor(2044) tensor(2013) 0 3 tensor(0.3340)
after the tensor(2044) tensor(1996) 0 4 tensor(0.4923)
after bank tensor(2044) tensor(2924) 0 5 tensor(0.2519)
after vault tensor(2044) tensor(11632) 0 6 tensor(0.2787)
after , tensor(2044) tensor(1010) 0 7 tensor(-0.0074)
after the tensor(2044) tensor(1996) 0 8 tensor(0.5428)
after bank tensor(2044) tensor(2924) 0 9 tensor(0.2962)
after robber tensor(2044) tensor(27307) 0 10 tensor(0.3771)
after was tensor(2044) tensor(2001) 0 11 tensor(0.5545)
after seen tensor(2044) tensor(2464) 0 12 tensor(0.4934)
after fishing tensor(2044) tensor(5645) 0 13 tensor(0.2893)
after on tensor(2044) tensor(2006) 0 14 tensor(0.3521)
after the tensor(2044) tensor(1996) 0 15 tensor(0.4320)
after mississippi tensor(2044) tensor(5900) 0 16 tensor(0.2139)
after river tensor(2044) tensor(2314) 0 17 tensor(0.2551)
after bank tensor(2044) te